# =====概要=====
こんな人が対象：機械学習初心者。モデル構築が良くわからない人。英語ばかりで困っている人。

コンペの内容からモデル作成までを解説します。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ライブラリをインポートします。

pandas：表計算

numpy：各種計算

matplotlib.pyplot：グラフ化

これらはほぼ確実に使うライブラリなのでとりあえずインポートしてもいいくらいです。

In [ ]:
train = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/train.csv")
print(train.shape)
train.head()

pandasの.read_csvでCSVファイルを読み込みます。

編集画面右端の”Data”というところからパスをコピーできるので（）に入れましょう。

データは以下の通り。

StudyInstanceUID：学習や予測に使う画像のID

ETT - Abnormal ～ Swan Ganz...：今回予測するラベル（０か１かを予測）

PatientID：患者のID。同じ患者の画像が何枚かある。

In [ ]:
import cv2
studyuid = train["StudyInstanceUID"][0]
path = "../input/ranzcr-clip-catheter-line-classification/train" + "/" + studyuid + ".jpg"
image = cv2.imread(path)
print(image.shape)

画像を読み込みますのでcv2をインポートしましょう。

画像のIDを１つだけ取り出してパスを作ります。trainのフォルダに画像が入っています。

これをcv2の.imreadに入れると画像データとして吐き出します。縦2048横2500色が3チャネルの画像です。

In [ ]:
plt.imshow(image)
plt.show()

matplotlibの.imshowで画像を表示できます。レントゲン写真です。

In [ ]:
train.loc[train["StudyInstanceUID"] == studyuid]

ここでもう一度学習データをみてみると、"NGT - Normal"に１が入っています。

"NGT"という種類のカテーテルが"Normal"つまり正常な位置にある画像ということです。

In [ ]:
studyuid = train["StudyInstanceUID"][3]
train.loc[train["StudyInstanceUID"] == studyuid]

３枚目の画像では"CVC - Abnormal"に１が入っています。

これはつまり"CVC"がアカン位置にあるという意味です。

In [ ]:
path = "../input/ranzcr-clip-catheter-line-classification/train" + "/" + studyuid + ".jpg"
image = cv2.imread(path)
plt.imshow(image)
plt.show()

さっきと同じように画像を読み込みました。アカン位置にあるみたいですがよくわかりません。

In [ ]:
annotations = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/train_annotations.csv")
print(annotations.shape)
annotations.head()

ここで使えるのがもう１つのデータである"train_annotation.csv"です。

画像のIDとラベル、そして画像上のxy座標データがあります。

In [ ]:
studyuid = annotations["StudyInstanceUID"][0]
annotations.loc[annotations["StudyInstanceUID"] == studyuid]

annotationの中で最初の画像IDを取り出しました。

同じ画像を取り扱う行が２つあります。

In [ ]:
annotations.loc[annotations["StudyInstanceUID"] == studyuid]["data"][0]

ここでxy座標を表すdataを見てみます。[x, y]の座標がいくつか入っていますが、これらは実は文字列です。

In [ ]:
import ast
points = annotations.loc[annotations["StudyInstanceUID"] == studyuid]["data"][0]
points = ast.literal_eval(points)
points = np.array(points)
points

astというライブラリをインポートして文字列をリストに変換します。

そのあとnp.array()でnumpy配列に変換しましょう。

In [ ]:
path = "../input/ranzcr-clip-catheter-line-classification/train" + "/" + studyuid + ".jpg"
image = cv2.imread(path)
plt.imshow(image)
plt.scatter(points[:, 0], points[:, 1])
plt.show()

対象の画像はimreadとimshowで表示できます。

.scatterでdataの座標を散布図としてプロットしましょう。[:, 0]は全行１列目を意味します。

すると青点でカテーテルの位置が出ました。

In [ ]:
annotations.loc[annotations["StudyInstanceUID"] == studyuid]

この画像は２つの正常な位置にあるカテーテルを取り扱っているのでどちらもプロットしましょう。

In [ ]:
def get_points(studyuid, num):
    points = annotations.loc[annotations["StudyInstanceUID"] == studyuid]["data"].values[num]
    points = ast.literal_eval(points)
    points = np.array(points)
    return points

path = "../input/ranzcr-clip-catheter-line-classification/train" + "/" + studyuid + ".jpg"
image = cv2.imread(path)
plt.imshow(image)
for i in range(annotations.loc[annotations["StudyInstanceUID"] == studyuid].shape[0]):
    points = get_points(studyuid, i)
    plt.scatter(points[:, 0], points[:, 1])
plt.show()

座標を読み取る処理は関数にまとめました。やっていることはさっきと同じです。

２つのカテーテルをプロットしました。

どちらも"CVC - Normal"つまり"CVC"というカテーテルが正常な位置にあることを意味します。これが正常らしいです。

In [ ]:
annotations.loc[annotations["label"] == "NGT - Abnormal"]

じゃあアカン位置にあるカテーテルはどんなのやということで、"Abnormal"を見つけます。

In [ ]:
studyuid = annotations["StudyInstanceUID"][1287]
annotations.loc[annotations["StudyInstanceUID"] == studyuid]

1287行目のデータに"Abnormal"があるので取り出しました。

In [ ]:
path = "../input/ranzcr-clip-catheter-line-classification/train" + "/" + studyuid + ".jpg"
image = cv2.imread(path)
plt.imshow(image)
for i in range(annotations.loc[annotations["StudyInstanceUID"] == studyuid].shape[0]):
    points = get_points(studyuid, i)
    label = annotations.loc[annotations["StudyInstanceUID"] == studyuid]["label"].values[i]
    plt.scatter(points[:, 0], points[:, 1], label = label)
plt.legend()
plt.show()

これまでと同じく画像と座標をプロットしました。

"Noamal"と"Abnormal"があるのでラベルも読み取ります。

青が"Abnormal"なのでアカン位置にあります。

何がダメなのかわかりませんがそれはDiscussionなどで有識者が語ってくれます。

## ○総括
画像の中に正常な位置やボーダーライン、アカン位置にあるカテーテルが写っている

今回はその画像からカテーテルの位置が正しいのか予測する

# =====データジェネレーター=====
モデル作成の前にデータを正しく取り出さなければいけません。

この点がテーブルデータと画像データで大きな違いです。

In [ ]:
train.head()

まず確認ですが、"StudyInstanceUID"から画像を引っ張ってきて各種ラベルの０１を予測するのが目標です。

In [ ]:
DEBUG = True
if DEBUG:
    train = train.sample(n = train.shape[0] // 10).reset_index(drop = True)

データ数が多いと学習に時間がかかるので1/10に減らしました。

実際に学習するときはDEBUG = Falseにしましょう。

In [ ]:
train["path"] = "../input/ranzcr-clip-catheter-line-classification/train" + "/" + train["StudyInstanceUID"] + ".jpg"
train.head()

"StudyInstanceUID"から画像のパスを作りました。

In [ ]:
path = train["path"].values[0]
image = cv2.imread(path)
plt.imshow(image)
plt.show()

例えば１つ目のパスから読み込んだ画像は上図です。

これをモデルに与えて同じ行にあるラベルを正解データとして教えます。

In [ ]:
labels = [
    'ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
    'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal',
    'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
    'Swan Ganz Catheter Present'
]
train[labels]

ラベルのリストは頻繁に使うので変数にしましょう。

In [ ]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train, test_size = 0.1)
print(train.shape, valid.shape)

学習データと評価データに分けます。

In [ ]:
import tensorflow as tf
train_ds = tf.data.Dataset.from_tensor_slices((train["path"].values, train[labels].values))
valid_ds = tf.data.Dataset.from_tensor_slices((valid["path"].values, valid[labels].values))
next(iter(train_ds))

ここからデータを取り出す作業に移ります。

色々な方法がありますが今回はtensorflowのDataset.from_tensor_slicesを使います。

画像のパスとそれに対する正解データを渡しましょう。

iterとnextで次のデータが出力されます。

In [ ]:
path = next(iter(train_ds))[0]
path

出力されるのはさっき渡した画像のパスと正解データです。[０]がパスです。

In [ ]:
image = tf.io.read_file(path)
#image

パスを画像データに変換しないといけないのでio.read_fileで指定したパスのデータを取得します。

しかし出力されるのは謎の形式のデータです。

In [ ]:
image = tf.image.decode_jpeg(image, channels = 3)
image

これをdecode_jpegでjpeg画像のデータに変換しましょう。

In [ ]:
plt.imshow(image)
plt.show()

変換したデータを.imshowに入れると確かに画像として取り出せています。

In [ ]:
print(image.shape)

しかし画像データのサイズはバラバラでモデルに入れるときに不都合が生じます。なので統一しましょう。

In [ ]:
image = tf.image.resize(image, [300, 300])
print(image.shape)

.image.resizeで変換できます。今回は縦横300にしました。

In [ ]:
def preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [300, 300])
    return image, label

ここまでの一連の処理を関数にしましょう。

In [ ]:
train_ds = train_ds.map(preprocess)
valid_ds = valid_ds.map(preprocess)

.mapに作った関数を渡すとその処理を内側でしてくれます。

In [ ]:
next(iter(train_ds))[0].shape

このように処理が済んだ状態で出力されます。

In [ ]:
train_ds = train_ds.batch(32)
valid_ds = valid_ds.batch(32)

.batchでミニバッチごとにデータを取り出せます。今回は32個ずつにしました。

In [ ]:
next(iter(train_ds))[0].shape

このように１回で32個のデータを取り出せています。

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size = AUTOTUNE)
#valid_ds = valid_ds.prefetch(buffer_size = AUTOTUNE)

.prefetchをするとモデル学習に使えるようになります。

AUTOTUNEは私自身よくわかっていません。おまじないのように書いています。

評価データ(valid)の方には不要みたいです。

ここまでで画像とラベルを取り出すジェネレーターが作成できました。

# =====モデル作成=====
これからモデルを作成しますが、既に学習済みで優秀なEfficientNetというモデルが存在するので、利用しましょう。

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

tensorflow.keras.applicationsにEfficientNetがあります。

B0～B7までの８種類が存在しますが今回はB0にしました。

その他必要なのはデータを受け取るInputと３次元の画像データを１次元に変換するプーリング層、そして出力に使う全結合層です。

Modelは設計した層を実際のモデルに組み立てるために使います。

In [ ]:
effnet = EfficientNetB0(weights = "../input/tfkerasefficientnetimagenetnotop/efficientnetb0_notop.h5", include_top = False)
effnet.trainable = False

EfficientNetB0を呼び出しました。

学習済みモデルなので"どんなデータで学習したモデルなのか"を指定します。今回は"imagenet"というデータで学習済みのモデルです。

本来はweightsをネットを通してこのノートブックにダウンロードするのですが、当コンペの性質上ネット接続はOFFにしないといけません。

なのでDatasetにアップロードしてあるweightsをノートブックのData->inputに入れておき、そこから読み取るようにします。

include_topは最終出力層も利用するか指定します。今回は11種類のラベルとして出力したい(カスタマイズしたい)のでFalseにしました。

.trainableはこれから学習する際にEfficientNetの部分も学習させるかをしていします。今回はめんどくさいのでFalseです。

In [ ]:
def create_model(input_shape, output_shape):
    inp = Input(input_shape)
    x = effnet(inp)
    x = GlobalAveragePooling2D()(x)
    x = Dense(32, activation = "relu")(x)
    out = Dense(output_shape, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = out)
    model.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["AUC"])
    return model

関数としてモデルを作成しました。

まず最初は画像データを受け取るInputです。サイズは(300, 300, 3)になりますが関数の引数input_shapeを代用しました。

次に先程定義したEfficientNetに入れます。この部分である程度精度良い学習が進みます。

EfficientNetから出力される次元が３次元なのでPoolingで１次元に変換します。

１次元データを全結合層(Dense)に入れて学習させます。EfficientNetは.trainable = Falseにしてますが、この層は別なので学習が進みます。

最後にアウトプットしたい列数として出力します。この時の活性化関数はsigmoidにしましょう。確率分布(０～１)にスケーリングされます。

Model(入力, 出力)と書いてこれまでに定義した流れをモデル化します。

.compileは学習の最適化アルゴリズム(optimizer)と損失関数(loss)、評価指標(metrics)を定義します。

In [ ]:
model = create_model(input_shape = (300, 300, 3), output_shape = len(labels))
model.summary()

実際にモデルを作成します。入力サイズはresizeしたので全て(300, 300, 3)です。出力はラベルの総数。

.summary()でモデルの中身を確認できます。

EfficientNetからPoolingで次元が減ってDenseに２回通しています。

In [ ]:
history = model.fit(train_ds, validation_data = valid_ds, epochs = 5)

.fitで学習できます。出力を変数で受け取っておくと学習過程を後で可視化できます。

入れるデータは学習データ(train_ds)と評価データ(valid_ds)、学習回数(epochs)です。

他にも条件を入れることがありますが長くなるので簡単なことだけにしました。

学習にはGPUを使うことをおススメします。CPUだと気が狂うほど長くなります。

無料で週に約40時間利用できるのでうまくマネジメントしましょう。

In [ ]:
model.evaluate(valid_ds)

.evaluateでモデル性能を確認できます。

出力されるのは[損失関数, 評価指標]です。

今回はbinary_crossentropy(小さいほど良い)とauc(大きいほど良い)ですね。

# =====提出=====

In [ ]:
test = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/sample_submission.csv")
test.shape

サンプルの提出データを取り出しました。

In [ ]:
test["path"] = "../input/ranzcr-clip-catheter-line-classification/test" + "/" + test["StudyInstanceUID"] + ".jpg"
test_ds = tf.data.Dataset.from_tensor_slices((test["path"].values, test[labels].values))
test_ds = test_ds.map(preprocess)
test_ds = test_ds.batch(64)
#test_ds = test_ds.prefetch(buffer_size = AUTOTUNE)

パスを作って画像を出力するデータジェネレーターを作りましょう。

これまでに解説した方法とおなじです。

In [ ]:
pred = model.predict(test_ds, verbose = 1)

.predictで予測ができます。

In [ ]:
pred

In [ ]:
pred.shape

予測値のサイズを見てみましょう。確かに11列です。

In [ ]:
submit = pd.DataFrame(pred, columns = labels)
submit.head()

提出用のデータにします。列名はlabelsがそのまま使えます。

In [ ]:
submit["StudyInstanceUID"] = test["StudyInstanceUID"]
submit = pd.concat([submit.iloc[:, -1], submit.iloc[:, :-1]], axis = 1)
submit.head()

画像IDが１列目に必要なのでtestから引っ張ってきます。

IDを１列目にしないとダメなのでilocでの切り取りとconcatで調整しました。

In [ ]:
submit.to_csv("submission.csv", index = False)

.to_csvでCSVファイルとして取り出せます。

index = Falseにしないと余計な列ができるので注意しましょう。